# Degradation and Acceleration Factors
This tool will provide a simple method for estimating degradation and for calculating acceleration factors. It interfaces with the degradation database to simplify acquisition of the degradation parameters.

**Requirements**:
- compatible weather file (e.g., PSM3, TMY3, EPW...)
- Accelerated testing chamber parameters
    - chamber irradiance [W/m^2]
    - chamber temperature [C]
    - chamber humidity [%]
    - & etc.
- Activation energies for test material [kJ/mol]
- Other degradation parameters

**Objectives**:
1. Read in the weather data
2. Gather basic degradation modeling data for a material of interest
3. Calculate absolute degradation rate
4. Run Monte Carlo simulation at a single site
5. Generate chamber or field data for environmental comparison
6. Calculate degradation acceleration factor of field location to chamber (or another location)
7. Run Monte Carlo simulation of the acceleration factor for a single site
8. Select a region of interest and specific data points
9. Produce a map of acceleration factors for a region

In [ ]:
# if running on google colab, uncomment the next line and execute this cell to install the dependencies and prevent "ModuleNotFoundError" in later cells:
# !pip install pvdeg==0.4.2

In [ ]:
import os
import pvdeg
import pandas as pd
from pvdeg import DATA_DIR
import json
from IPython.display import display, Math

import pvlib
print(pvlib.__version__)
from pvlib import iotools

In [ ]:
# This information helps with debugging and getting support :)
import sys, platform
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("pvdeg version ", pvdeg.__version__)
print(DATA_DIR)

## 1. Read In the Weather Data

The function has these minimum requirements when using a weather data file:
- Weather data containing (at least) DNI, DHI, GHI, Temperature, RH, and Wind-Speed data at module level.
- Site meta-data containing (at least) latitude, longitude, and time zone

Alternatively one may can get meterological data from the NSRDB or PVGIS with just the longitude and latitude. This function for the NSRDB (via NSRDB 'PSM3') works primarily for most of North America and South America. PVGIS works for most of the rest of the world (via SARAH 'PVGIS'). See the tutorial "Weather Database Access.ipynb" tutorial on PVdeg or Jensen et al. https://doi.org/10.1016/j.solener.2023.112092 for satellite coverage information.

In [ ]:
# Get data from a supplied data file (Do not use the next box of code if using your own file)
weather_file = os.path.join(DATA_DIR, 'psm3_demo.csv')
weather_df, meta = pvdeg.weather.read(weather_file,'csv',find_meta=False)
print(weather_file)
print(meta)

meta = pvdeg.weather.map_meta(meta)
meta = pvdeg.weather.find_metadata(meta)
print(meta)

In [ ]:
# This routine will get a meteorological dataset from anywhere in the world where it is available
#weather_id = (24.7136, 46.6753) #Riyadh, Saudi Arabia
#weather_id = (35.6754, 139.65) #Tokyo, Japan
#weather_id = (-43.52646, 172.62165) #Christchurch, New Zealand
#weather_id = (64.84031, -147.73836) #Fairbanks, Alaska
#weather_id = (65.14037, -21.91633) #Reykjavik, Iceland
weather_id = (33.4152, -111.8315) #Mesa, Arizona
#weather_id = (0,0) # Somewhere else you are interested in.
weather_df, meta = pvdeg.weather.get_anywhere(id=weather_id)
print(meta)
#display(weather_df)

#### POA Irradiance
Next we need to calculate the stress parameters including temperature and humidity. We start with POA irradiance.
Irradiance_kwarg governs the array orientation for doing the POA calculations. 
It is defaulted to a north-south single axis tracking. A fixed tilt set of parameters is included but is blocked out. 
Look in spectral.py and/or PVLib here for 1-axis kwargs, 
https://pvlib-python.readthedocs.io/en/v0.7.2/generated/pvlib.tracking.singleaxis.html#pvlib.tracking.singleaxis  
and for fixed tilt,  
https://pvlib-python.readthedocs.io/en/v0.7.2/generated/pvlib.irradiance.gti_dirint.html?highlight=poa .  

In [ ]:
#irradiance_kwarg ={
    #"tilt": None,
    #"azimuth": None,
    #"module_mount": 'fixed'}
irradiance_kwarg ={
    "axis_tilt": None,
    "axis_azimuth": None,
    "module_mount": '1_axis'}
poa_df = pvdeg.spectral.poa_irradiance(weather_df=weather_df, meta=meta, **irradiance_kwarg)
#display(poa_df)


#### Get Spectrally Resolved Irradiance Data
This first set of commands will calculate spectrally resolved irradiance data. This may or may not be needed for a given degradation model and can be skipped here. 

In [ ]:
# this whole block needs to be replaced with call to calculate spectrally resolved irradiance.

from pvdeg import TEST_DATA_DIR
INPUT_SPECTRA = os.path.join(TEST_DATA_DIR, r"spectra_pytest.csv")
data = pd.read_csv(INPUT_SPECTRA)
display(data)
print(INPUT_SPECTRA)


#### Cell or Module Surface Temperature
The following will calculate the cell and module surface temperature using the King model as a default. Other models can be used as described at,  
https://pvlib-python.readthedocs.io/en/stable/reference/pv_modeling/temperature.html. The difference is less than one °C for ground mounted systems
but can be as high as 3 °C for a high temperature building integrated system.

Here the temperatures are added to the dataframe and the module temperature is selected as the default 'temperatue' for the degradation calculations. If it is a cell degradation that is being investigated, temp_cell should be used.

In [ ]:
temp_cell = pvdeg.temperature.cell(weather_df=weather_df, meta=meta, poa=poa_df)
temp_module = pvdeg.temperature.module(weather_df=weather_df, meta=meta, poa=poa_df)

weather_df['temp_cell'] = temp_cell
weather_df['temp_module'] = temp_module

weather_df['temperature'] = weather_df['temp_module']
#weather_df['temperature'] = weather_df['temp_cell']

#### Humidity
Depending on the component for which the calculation is being run on, the desired humidity may be the atmospheric humidity, the module surface humidity, the humidity in front of a cell with a permeable backsheet, the humidity in the backsheet, the humidity in the back encapsulant or another custom humidity location such as a diffusion limited location. The folowing are options for doing all of these calculations. Here all the different humidities are put in the weather_df dataframe, but to select one to be specifically used it should be named 'RH' for most degradation functions (check the documentation of a specific degradation calculation if in doubt). Here the surface humidity is selected as a default.

In [ ]:
# Calculate relative humidity at different locations in the module
RH_surface = pvdeg.humidity.surface_outside(weather_df['relative_humidity'], weather_df['temp_air'],temp_module)
RH_front_encapsulant = pvdeg.humidity.front_encap(weather_df['relative_humidity'], weather_df['temp_air'],temp_cell,encapsulant='W001')
RH_back_encapsulant = pvdeg.humidity.Ce(
    temp_module = temp_module,
    rh_surface = RH_surface,
    backsheet='W017',
    encapsulant='W001')
Ce_back_encapsulant = pvdeg.humidity.Ce(
    temp_module = temp_module,
    rh_surface = RH_surface,
    backsheet='W017',
    encapsulant='W001',
    output='ce')
RH_backsheet = (RH_surface + RH_back_encapsulant) / 2



Append the calculated values into the weather DataFrame.
Note: putting the values into the weather_df DataFrame is not strictly necessary, but may be convenient for later use in the degradation calculations.

In [ ]:
weather_df['RH_surface'] = RH_surface
weather_df['RH_front_encapsulant'] = RH_front_encapsulant
weather_df['Ce_back_encapsulant'] = Ce_back_encapsulant
weather_df['RH_back_encapsulant'] = RH_back_encapsulant
weather_df['RH_backsheet'] = RH_backsheet

weather_df['poa_global'] = poa_df['poa_global']

Each of the necessary arrays of data can be individually sent to a function for calculation in the function call, or they can be combined into a single dataframe. The degradation functions are set up to first check for a specific data set in the function call but if not found it looks for specific data or a suitable substitute in the weather dataframe.

You can select one of the RH values to be used as the relative humidity in the degradation model calculations by assigning it to to column "RH" in the dataframe.
Alternatively, the "RH" data can be sent to the degradation function explicitly in the function call.

In [ ]:
weather_df['RH'] = RH_surface
#weather_df['RH'] =RH_front_encapsulant
#weather_df['RH'] = Ce_back_encapsulant
#weather_df['RH'] = RH_back_encapsulant
#weather_df['RH'] = RH_backsheet

#display(weather_df)

## 2. Gather Basic Degradation Modeling Data for a Material of Interest

First we need to gather in the parameters for the degradation process of interest. This includes things such as the activiation energy and parameters defining the sensitivity to moisture, UV light, voltage, and other stressors.
For this tutorial we will need solar position, POA, PV cell and module temperature. Let's gernate those individually with their respective functions.
The blocked out text will produce a list of key fields from the database for each entry.

In [ ]:
#kwarg_variables = pvdeg.utilities._read_material(name=None, fname="DegradationDatabase", item=("Material", "Equation", "KeyWords", "EquationType"))
#print(json.dumps(kwarg_variables, skipkeys = True, indent = 0 ).replace("{" + "\n", "{").replace('\"' + "\n", "\"").replace(': {' , ':' + "\n" + "{").replace('},' + "\n", '},' +'\n' +'\n'))
pvdeg.utilities.display_json(pvdeg_file="DegradationDatabase", fp=DATA_DIR)

This next set of codes will take the data from the extracted portion of the Json library and create a list of variables from it. If more variables need to be modified or added, this is where it should be done.

In [ ]:
deg_data = pvdeg.utilities.read_material(fp=DATA_DIR, key="D036", pvdeg_file="DegradationDatabase")
print(json.dumps(deg_data, skipkeys = True, indent = 0 ).replace("{" + "\n", "{").replace('\"' + "\n", "\"").replace(': {' , ':' + "\n" + "{").replace('\n'+'}', '}'))

Here we pull out the relevant equation code identifier needed for running the calculations.

In [ ]:
func = "pvdeg.degradation." + deg_data["EquationType"]
print(func)
display(Math("\\Large " + deg_data["Equation"]))

## 3. Calculate Absolute Degradation Rate

To do this calculation, we must have degradation parameter data for a process that is complete with all the necessary variables. 

In [ ]:
from pvdeg import TEST_DATA_DIR
INPUT_SPECTRA = os.path.join(TEST_DATA_DIR, r"spectra_pytest.csv")
data = pd.read_csv(INPUT_SPECTRA)
display(data)
print(INPUT_SPECTRA)



## 3. VantHoff Degradation

Van 't Hoff Irradiance Degradation

For one year of degredation the controlled environmnet lamp settings will need to be set to IWa.

As with most `pvdeg` functions, the following functions will always require two arguments (weather_df and meta)

## 4. Arrhenius
Calculate the Acceleration Factor between the rate of degredation of a modeled environmnet versus a modeled controlled environmnet

Example: "If the AF=25 then 1 year of Controlled Environment exposure is equal to 25 years in the field"

Equation:
$$ AF = N * \frac{ I_{chamber}^x * RH_{chamber}^n * e^{\frac{- E_a}{k T_{chamber}}} }{ \Sigma (I_{POA}^x * RH_{outdoor}^n * e^{\frac{-E_a}{k T_outdoor}}) }$$

Function to calculate IWa, the Environment Characterization (W/m²). For one year of degredation the controlled environmnet lamp settings will need to be set at IWa.

Equation:
$$ I_{WA} = [ \frac{ \Sigma (I_{outdoor}^x * RH_{outdoor}^n e^{\frac{-E_a}{k T_{outdood}}}) }{ N * RH_{WA}^n * e^{- \frac{E_a}{k T_eq}} } ]^{\frac{1}{x}} $$

In [ ]:
# relative humidity within chamber (%)
rh_chamber = 15
# arrhenius activation energy (kj/mol)
Ea = 40
I_chamber = 1000  # irradiance within chamber (W/m^2)
temp_chamber = 25  # temperature within chamber (C)

rh_surface = pvdeg.humidity.surface_outside(rh_ambient=weather_df['relative_humidity'],
                                               temp_ambient=weather_df['temp_air'],
                                               temp_module=temp_module)

arrhenius_deg = pvdeg.degradation.arrhenius_deg(weather_df=weather_df, meta=meta,
                                                rh_outdoor=rh_surface,
                                                I_chamber=I_chamber,
                                                rh_chamber=rh_chamber,
                                                temp_chamber=temp_chamber,
                                                poa=poa_df,
                                                temp=temp_cell,
                                                Ea=Ea)

irr_weighted_avg_a = pvdeg.degradation.IwaArrhenius(weather_df=weather_df, meta=meta,
                                                    poa=poa_df,
                                                    rh_outdoor=weather_df['relative_humidity'],
                                                    temp=temp_cell,
                                                    Ea=Ea)

## 5. Quick Method (Degradation)

For quick calculations, you can omit POA and both module and cell temperature. The function will calculate these figures as needed using the available weather data with the default options for PV module configuration.

In [ ]:
# chamber settings
I_chamber= 1000
temp_chamber=60
rh_chamber=15

# activation energy
Ea = 40

vantHoff_deg = pvdeg.degradation.vantHoff_deg(weather_df=weather_df, meta=meta,
                                              I_chamber=I_chamber,
                                              temp_chamber=temp_chamber)

irr_weighted_avg_v = pvdeg.degradation.IwaVantHoff(weather_df=weather_df, meta=meta)

In [ ]:
rh_surface = pvdeg.humidity.surface_outside(rh_ambient=weather_df['relative_humidity'],
                                               temp_ambient=weather_df['temp_air'],
                                               temp_module=temp_module)

arrhenius_deg = pvdeg.degradation.arrhenius_deg(weather_df=weather_df, meta=meta,
                                                rh_outdoor=rh_surface,
                                                I_chamber=I_chamber,
                                                rh_chamber=rh_chamber,
                                                temp_chamber=temp_chamber,
                                                Ea=Ea)

irr_weighted_avg_a = pvdeg.degradation.IwaArrhenius(weather_df=weather_df, meta=meta,
                                                    rh_outdoor=weather_df['relative_humidity'],
                                                    Ea=Ea)

## 6. Solder Fatigue

Estimate the thermomechanical fatigue of flat plate photovoltaic module solder joints over the time range given using estimated cell temperature. Like other `pvdeg` funcitons, the minimal parameters are (weather_df, meta). Running the function with only these two inputs will use default PV module configurations ( open_rack_glass_polymer ) and the 'sapm' temperature model over the entire length of the weather data. 

In [ ]:
fatigue = pvdeg.fatigue.solder_fatigue(weather_df=weather_df, meta=meta)

If you wish to reduce the span of time or use a non-default temperature model, you may specify the parameters manually. Let's try an explicit example.
We want the solder fatigue estimated over the month of June for a roof mounted glass-front polymer-back module.

1. Lets create a datetime-index for the month of June.
2. Next, generate the cell temperature. Make sure to explicity restrict the weather data to our dt-index for June. Next, declare the PV module configuration.
3. Calculate the fatigue. Explicity specify the time_range (our dt-index for June from step 1) and the cell temperature as we caculated in step 2

In [ ]:
# select the month of June
time_range = weather_df.index[weather_df.index.month == 6]

# calculate cell temperature over our selected date-time range.
# specify the module configuration
temp_cell = pvdeg.temperature.cell(weather_df=weather_df.loc[time_range], meta=meta,
                                   temp_model='sapm',
                                   conf='insulated_back_glass_polymer')


fatigue = pvdeg.fatigue.solder_fatigue(weather_df=weather_df, meta=meta,
                                       time_range = time_range,
                                       temp_cell = temp_cell)